In [ ]:
import requests
import urllib3
import json
import os
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
cd = os.path.abspath(os.getcwd())
with open(f'{cd}/config_personal.json') as f:
    data = json.load(f)

In [ ]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"
payload = data['payload']

In [ ]:
res = requests.post(auth_url, data=payload, verify=False)
header = {'Authorization': 'Bearer ' + res.json()['access_token']}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [ ]:
import pandas as pd
import seaborn as sns
# seaborn is actually built on top of matplotlib
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [ ]:
activities = pd.json_normalize(my_dataset)
pd.set_option('display.max_columns', 83)
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.max_rows', 25)
pd.set_option("display.precision", 2)

In [ ]:
activities = activities.rename(columns={'average_speed' : 'average_speed_mps', 
                                        'max_speed' : 'max_speed_mps',
                                        'moving_time' : 'moving_time_s',
                                        'elapsed_time' : 'elapsed_time_s'})

# fix 'start_date_local' and create 2 new columns 'start_time' and 'moving_time(min)'
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.strftime("%d/%m/%y")

# seconds to minutes
activities['moving_time(min)'] = activities['moving_time_s'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%H:%M:%S'))

# meters to kms
activities['distance'] = activities[
    (activities['type'] == 'Run') |
    (activities['type'] == 'Walk')]['distance'] / 1000

# meters per second to kms per hour
activities['average_speed_mps'] = activities['average_speed_mps'] * (18/5)
activities['max_speed_mps'] = activities['max_speed_mps'] * (18/5)

# from (GMT+01:00) Europe/Berlin -> Europe/Berlin
activities['timezone'] = activities['timezone'].str.split(' ').str[-1]

# rename converted columns
activities = activities.rename(columns={'average_speed_mps': 'average_speed_kmh',
                                        'max_speed_mps' : 'max_speed_kmh',
                                        'distance' : 'distance_km'})

In [ ]:
activities

In [ ]:
cols = ['upload_id', 'name', 'type', 'distance_km', 'moving_time(min)','start_time',  'start_date_local',  'timezone',
        'average_speed_kmh', 'max_speed_kmh', 'total_elevation_gain', 'average_heartrate',
        'max_heartrate', 'achievement_count', 'kudos_count', 'visibility',  ]
activities = activities[cols]

In [ ]:
# usage -> 25.1 KB to 22.6 KB
activities['visibility'] = pd.Categorical(activities['visibility'])
activities['timezone'] = pd.Categorical(activities['timezone'])

In [ ]:
activities

In [ ]:
activities.shape

 IDEAS
 check the name: 'run'-> move to a new df['running']
                'other' -> df['rest_sessions']
activities

In [ ]:
activities['type'].value_counts()

In [ ]:
run = activities.loc[activities['type'] == 'Run']

sns.set(style="ticks", context="talk")
sns.regplot(x='distance', y='average_speed', data=run).set_title("Average Speed vs Distance")

## sns.set(style="ticks", context="talk")
sns.regplot(x='distance', y='max_speed', data=run).set_title("Max Speed vs Distance")

fig = plt.figure() # overall container
ax1 = fig.add_subplot(111) # add 1 by 1 plot to the figure
x = np.asarray(run.start_date_local) # convert data to numpy array
y = np.asarray(run.average_speed)
ax1.plot_date(x, y)
ax1.set_title('Average Speed over Time')

x2 = mdates.date2num(x) # add a trend line
z = np.polyfit(x2, y, 1)
p = np.poly1d(z)
plt.plot(x, p(x2), 'r--')

fig.autofmt_xdate(rotation=45) # format the figure and display
fig.tight_layout()
fig.show()

fig